In [2]:
import cv2 
import os 
import plotly
import kaleido
import plotly.graph_objects as go
import plotly.io as pio
import pandas as pd 
import gpxpy
import requests
import math
from PIL import Image

In [3]:
course = "Ronde van Vlaanderen 1"
alias = "2023_RVV_1"
vid_path = "/Users/senneloobuyck/Documents/Thesis_project/01_inputs/videos/2023 Ronde van Vlaanderen 1.mp4"
heli_path = "/Users/senneloobuyck/Documents/Thesis_project/01_inputs/csv-files/2023 Ronde van Vlaanderen 1 heli.csv"
gpx_path = "/Users/senneloobuyck/Documents/Thesis_project/01_inputs/gpx-files/2023 Ronde van Vlaanderen course.gpx"

# course = "Kuurne Brussel Kuurne"
# alias = "2020_KBK"
# vid_path = "/Users/senneloobuyck/Documents/Thesis_project/01_inputs/videos/2020 KBK.mp4"

In [4]:
# read in helicopter data
heli_df = pd.read_csv(heli_path)

heli_df['lat'] = heli_df.apply(lambda row: float(row['lat']) * 1 if row['NS'] == 'N' else float(row['lat']) * (-1), axis=1)
heli_df['lng'] = heli_df.apply(lambda row: float(row['lng']) * 1 if row['EW'] == 'E' else float(row['lng']) * (-1), axis=1)

# make new df with only lat, lng, alt and seconds_from_start
heli_df = heli_df[['lat', 'lng', 'alt', 'seconds_from_start']]

# calculate pitch

print(heli_df.head())

         lat       lng     alt  seconds_from_start
0  50.775510  3.525282  157.52                 0.0
1  50.775488  3.525138  157.78                 0.0
2  50.775477  3.524995  157.80                 1.0
3  50.775475  3.524853  157.57                 2.0
4  50.775483  3.524715  157.21                 2.0


In [5]:
# read in course data
# Parse GPX file
gpx_file = open(gpx_path, 'r')
gpx = gpxpy.parse(gpx_file)

# extract latitude and longitude coordinates for each point
route = [(point.latitude, point.longitude) for track in gpx.tracks for segment in track.segments for point in segment.points]
route_df = pd.DataFrame(route, columns=['latitude', 'longitude'])

print(route_df.head())

    latitude  longitude
0  51.208498   3.224938
1  51.208498   3.224938
2  51.208498   3.224938
3  51.208477   3.224981
4  51.208456   3.225002


In [6]:
def get_intersection_coordinates(lat, lon) : 
    # some lists and dataframes to store the data
    nodes = []
    intersection_nodes = []
    df_nodes = pd.DataFrame(columns=['id', 'lat', 'lon'])
    df_intersections = pd.DataFrame(columns=['lat', 'lon'])

    overpass_url = "http://overpass-api.de/api/interpreter"


    # first get the ways around the point 
    # .w is a temporary variable that stores the result of the first query
    # node(way_cnt:2-);   // select those nodes that are members of two or more of the given ways
    overpass_query = f"""[out:json];
                        way(around: 300, {lat},{lon})["highway"]->.w;
                        node(way_cnt.w:2-);  
                        out;"""

    response = requests.get(overpass_url, params={'data': overpass_query})

    for item in response.json()['elements']:
        df_intersections.loc[len(df_intersections)] = {'lat': item['lat'], 'lon': item['lon']}


    # print(df_intersections)
    return df_intersections

In [7]:
mapbox_access_token = "pk.eyJ1Ijoic2VubmVsb29idXljayIsImEiOiJjbG1zeDVreHowZmw5Mm1xY2lxeTFtYTB2In0.ymj-xZAfeeivbksi1PJttw"

In [8]:
def processing_frames_without_detection(path, alias, time_interval, course, heli_df, route_df) :
    print(f"Processing video frames of {course}")
    # create video capture object
    cap = cv2.VideoCapture(path)

    # set frame rate of video, property of the video itself
    fps = cap.get(cv2.CAP_PROP_FPS)
    print(f"Video has a fps of {fps} frames per second. ")

    # set time interval (in seconds) for frame capture
    interval = time_interval

    # calculate frame interval based on fps and time interval
    # frame interval in #frames
    frame_interval = int(fps * interval)

    # initialize frame counter
    frame_count = 0

    mapbox_route = go.Scattermapbox(
        lon=route_df['longitude'],
        lat=route_df['latitude'],
        name="Route points",
        mode="markers",
        marker=dict(
            color='red',
            size=8,
        ),
    )

    # loop over frames in video
    while cap.isOpened():
        # read next frame from video
        ret, frame = cap.read()

        # check if frame was read successfully
        if not ret:
            break

        # check if it's time to capture a frame
        if frame_count % frame_interval == 0:
            seconds = frame_count / fps
    

            mapbox_heli_other = go.Scattermapbox(
                lon=heli_df['lng'],
                lat=heli_df['lat'],
                mode='markers',
                marker=dict(
                    color='blue',
                    size=8,
                ),
                name="Other Helicopter points"
            )

            if seconds in heli_df['seconds_from_start'].values:
                filtered_df = heli_df[heli_df['seconds_from_start'] == seconds]

            # print(f"filtered_df: {filtered_df}")
            filtered_df_first = filtered_df.iloc[0]

            mapbox_heli_now = go.Scattermapbox(
                lon=filtered_df['lng'],
                lat=filtered_df['lat'],
                mode='markers',
                marker=dict(
                    color='yellow',
                    size=8,
                ),
                name="Live Helicopter point(s)"
            )

            intersec_df = get_intersection_coordinates(filtered_df_first['lat'], filtered_df_first['lng'])

            mapbox_intersec= go.Scattermapbox(
                lon=intersec_df['lon'],
                lat=intersec_df['lat'],
                # mode = 'markers',
                mode='markers',
                marker=dict(
                    color='black',
                    size=8,
                ),
                name="Intersections",
                textposition="top right",
                textfont=dict(
                    color='black',
                    size=8,
                )
            )

            # now set up the layout of the map
            layout = go.Layout(
                width=1200,
                height=900,
                mapbox=dict(
                    accesstoken=mapbox_access_token,
                    style="open-street-map",
                    zoom=16,
                    center=dict(
                        lon=filtered_df.iloc[0]['lng'],
                        lat=filtered_df.iloc[0]['lat']
                    )
                )
            )

            data = [mapbox_route, mapbox_heli_other, mapbox_heli_now, mapbox_intersec]

            fig = go.Figure(data=data, layout=layout)

            # fig.show()

            output_dir_maps = "/Users/senneloobuyck/Documents/Thesis_project/outputs/maps"
            output_dir_map_image = os.path.join(output_dir_maps, alias, f"map_{frame_count}.jpg")
            pio.write_image(fig, output_dir_map_image, format="jpg")

            # put map and frame with detection together in 1 image
            map_image = Image.open(output_dir_map_image)

            # Convert video frame from BGR to RGB
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

            # Create a PIL Image from the video frame
            frame_image = Image.fromarray(frame_rgb)

            # Combine the map image and video frame side by side
            combined_image = Image.new('RGB', (map_image.width + frame_image.width, map_image.height))
            combined_image.paste(map_image, (0, 0))
            combined_image.paste(frame_image, (map_image.width, 0))

            # Specify the output image file path for the combined image
            output_dir_combined_image = "/Users/senneloobuyck/Documents/Thesis_project/outputs/demo"
            output_combined_map_image = os.path.join(output_dir_combined_image, alias, f"combined_map_frame_{frame_count}.jpg")
            print(output_combined_map_image)

            # Save the combined image
            combined_image.save(output_combined_map_image)

        # increment frame counter
        frame_count += 1

    # release video capture object
    cap.release()

In [9]:
processing_frames_without_detection(vid_path, alias, 5, course, heli_df, route_df)

Processing video frames of Ronde van Vlaanderen 1
Video has a fps of 0.0 frames per second. 


OpenCV: Couldn't read video stream from file "/Users/senneloobuyck/Documents/Thesis_project/01_inputs/videos/2023 Ronde van Vlaanderen 1.mp4"


In [10]:
def get_map_from_seconds_passed(heli_df, route_df, seconds_passed) : 


    mapbox_route = go.Scattermapbox(
        lon=route_df['longitude'],
        lat=route_df['latitude'],
        name="Route points",
        mode="markers",
        marker=dict(
            color='red',
            size=8,
        ),
    )
    

    mapbox_heli_other = go.Scattermapbox(
        lon=heli_df['lng'],
        lat=heli_df['lat'],
        mode='markers',
        marker=dict(
            color='blue',
            size=8,
        ),
        name="Other Helicopter points"
    )

    if seconds_passed in heli_df['seconds_from_start'].values:
        filtered_df = heli_df[heli_df['seconds_from_start'] == seconds_passed]

    # print(f"filtered_df: {filtered_df}")
    filtered_df_first = filtered_df.iloc[0]
    print("Live helicopter point ")
    print(filtered_df_first)

    mapbox_heli_now = go.Scattermapbox(
        lon=filtered_df['lng'],
        lat=filtered_df['lat'],
        mode='markers',
        marker=dict(
            color='yellow',
            size=8,
        ),
        name="Live Helicopter point(s)"
    )

    intersec_df = get_intersection_coordinates(filtered_df_first['lat'], filtered_df_first['lng'])
    print("Intersections")
    print(intersec_df)

    mapbox_intersec= go.Scattermapbox(
        lon=intersec_df['lon'],
        lat=intersec_df['lat'],
        # mode = 'markers',
        mode='markers',
        marker=dict(
            color='black',
            size=8,
        ),
        name="Intersections",
        textposition="top right",
        textfont=dict(
            color='black',
            size=8,
        )
    )

    # now set up the layout of the map
    layout = go.Layout(
        width=1200,
        height=900,
        mapbox=dict(
            accesstoken=mapbox_access_token,
            style="open-street-map",
            zoom=16,
            center=dict(
                lon=filtered_df.iloc[0]['lng'],
                lat=filtered_df.iloc[0]['lat']
            )
        )
    )

    data = [mapbox_route, mapbox_heli_other, mapbox_heli_now, mapbox_intersec]

    fig = go.Figure(data=data, layout=layout)

    fig.show()

In [11]:
# frame_count = 1275
# fps = 25
# seconds_passed = frame_count / fps
seconds_passed = 366
print(seconds_passed)

get_map_from_seconds_passed(heli_df, route_df, seconds_passed)

366
Live helicopter point 
lat                    50.764010
lng                     3.541103
alt                   189.880000
seconds_from_start    366.000000
Name: 556, dtype: float64
Intersections
          lat       lon
0   50.765872  3.544358
1   50.763079  3.540643
2   50.764544  3.537733
3   50.765409  3.542850
4   50.762907  3.540710
5   50.766391  3.541413
6   50.765930  3.544066
7   50.766038  3.543485
8   50.766277  3.542130
9   50.765720  3.542603
10  50.765399  3.542799
11  50.764545  3.537678
12  50.767109  3.536825


In [25]:
def calculate_bearing(lat1, long1, lat2, long2) : 
    lat1 = math.radians(lat1)
    long1 = math.radians(long1)
    lat2 = math.radians(lat2)
    long2 = math.radians(long2)

    d_long = long2 - long1

    y = math.sin(d_long) * math.cos(lat2)
    x = math.cos(lat1) * math.sin(lat2) - math.sin(lat1) * math.cos(lat2) * math.cos(d_long)

    brng = math.atan2(y, x)

    brng = math.degrees(brng)
    brng = (brng + 360) % 360
    # brng = 360 - brng # count degrees clockwise - remove to make counter-clockwise
    return brng

In [26]:
heli_lat = 50.77718166666666
heli_lon = 3.52441
intersec_lat = 50.7786715
intersec_lon = 3.5252944 

In [27]:
bearing = calculate_bearing(heli_lat, heli_lon, intersec_lat, intersec_lon)
print(bearing)

20.574015630418444


In [28]:
def haversine(lat1, lon1, lat2, lon2) : 
    R = 6371
    d_lat = math.radians(lat2 - lat1)
    d_lon = math.radians(lon2 - lon1)
    lat1 = math.radians(lat1)
    lat2 = math.radians(lat2)
    a = math.sin(d_lat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(d_lon/2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    d = R * c * 1000
    return d


In [29]:
# calculate pitch/tilt
def calculate_pitch(heli_lat, heli_lon, intersec_lat, intersec_lon) : 
    dist = haversine(heli_lat, heli_lon, intersec_lat, intersec_lon)
    
    result = heli_df[(heli_df['lat'] == heli_lat) & (heli_df['lng'] == heli_lon)]
    if not result.empty:
        altitude = result.iloc[0]['alt']

    print(f"Distance: {dist}")
    print(f"Altitude: {altitude}")

    alpha = math.asin(altitude/dist)
    pitch = 90 - alpha
    return pitch 

In [30]:
pitch = calculate_pitch(heli_lat, heli_lon, intersec_lat, intersec_lon)
print(pitch)

Distance: 176.94821325677256
Altitude: 158.12
88.8947091553299


In [ ]:
def get_mapbox_intersection_image(lat, lon, bearing, tilt) : 
    lat = 50.7878
    lon = 3.5609
    zoom = 17.12
    bearing = 0
    tilt = 0
    width = 512
    height = 512
    access_token = "pk.eyJ1Ijoic2VubmVsb29idXljayIsImEiOiJjbGxjMjhucDkwY3cxM2RvMXpqcjB1Y3FnIn0.4G8xiPZ9ZI2_1xlUPmDJMA" 
    styles_url = "mapbox://styles/senneloobuyck/cllcojw8d00wy01pb6rqcdmm5"

    style_code = styles_url.split('/')[-1]

    url = f"https://api.mapbox.com/styles/v1/senneloobuyck/{style_code}/static/{str(lon)},{str(lat)},{str(zoom)},{str(bearing)},{str(tilt)}/{str(width)}x{str(height)}?access_token={access_token}"